# Final Project: Deep Neutral Network
## Model 1:  Predict Glacier Retreat with Tensor Flow's Keras

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

Note:  The raw data files have been cleaned and prepared for use in the

Jupyter Source File "Data_Cleaning_Glacier_Retreat".  The output for this file

is in the "resources" folder and includes data sets for each of the following:
1.  Environmental Parameters:  Global Temperature, Global Sea Rise, and Global CO2
        Filename:  env_parameters_1.csv

2.  Population, Economic, and Farming Parameters: World Population, Urban Population,
        investment, and cereal output production by acre.
        filename: pop_farm_parameters_2.csv

3.  Change in Temperature by Country:  Average delta T by country.
        filename: dT_Country_parameters_3.csv

4.  Change in Forestation by Country:  Percent change in forestation by country.
        filename: deforest_parameters_4.csv

---
### First Model: Glacier Retreat and Typical Climate Change Measurements.

In [2]:
# Review and load the dataset for Typical Climate Change Measurements
file_path = "resources\env_parameters_1.csv"
df_parameters_1 = pd.read_csv(file_path)

# Review the DataFrame
df_parameters_1.head()

,year,glacier_retreat,Arctic_DegC_Anomaly,Annual_Increase_PPM,ann_d_GSL_mm
0,1979,0,0.12,2.14,1.234363
1,1980,1,0.31,1.71,1.049035
2,1981,0,0.32,1.15,2.591535
3,1982,1,0.18,1.00,0.234687
4,1983,1,0.23,1.83,-1.269578


In [3]:
# Check the glacier retreat value counts
df_parameters_1["glacier_retreat"].value_counts()
# low quantity but evenly split

glacier_retreat
0    25
1    19
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “glacier_retreat”. The remaining columns should define the features dataset.

In [4]:
# Define the target set y using the glacier_retreat column
# Remember that .values creates a numpy array
y = df_parameters_1["glacier_retreat"].values
y

array([0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
      dtype=int64)

In [5]:
# Define features set X by selecting remaining columns, drop year and 
# glacier_retreat
X = df_parameters_1.drop(["year","glacier_retreat"],axis=1)

# Review the features DataFrame
display(X.head(3),X.tail(3))

,Arctic_DegC_Anomaly,Annual_Increase_PPM,ann_d_GSL_mm
0,0.12,2.14,1.234363
1,0.31,1.71,1.049035
2,0.32,1.15,2.591535


,Arctic_DegC_Anomaly,Annual_Increase_PPM,ann_d_GSL_mm
41,0.91,2.30,-1.376702
42,0.90,2.47,1.546960
43,0.90,2.17,6.370000


### Step 3: Split the features and target sets into training and testing datasets.


In [6]:
# Split the preprocessed data into a training and testing dataset
# Reduced test size due to small data set  First Pass: 0.3
# 2nd Pass model_1a.keras:  test_size=0.2
# 3rd Pass model_1b.keras:  test_size=0.5
# The best result was first pass .03, so this model is released with that value.
# Assign the function a random_state equal to 13
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [7]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Scale the Data <<< added by student
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.



In [8]:
# Define the the number of inputs (features) to the model (old value was 11)
number_input_features = len(X_train.columns)
# Review the number of features
number_input_features

3

In [9]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  2
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 1
# Define the number of neurons in the output layer
nn_output_layer = 1

In [10]:
# Create the Sequential model instance
nn = tf.keras.models.Sequential()

# Add the first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
                             input_dim=number_input_features,activation="relu"))

# Add the second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons
# and activation function
nn.add(tf.keras.layers.Dense(units=nn_output_layer, activation="sigmoid"))

c:\Users\frisb\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Display the Sequential model summary
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 2)              │             8 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             3 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13 (52.00 B)

 Trainable params: 13 (52.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [12]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

In [13]:
# Fit the model using 50 epochs and the training data
# Note: first Effort 50 epochs = .6667.  Changed to 100 epochs
#Note:  second effort 100 epochs = .7667 loss =.5974
# Changed to 200 epochs.  This did not improve.  Changed back to 100 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 798ms/step - accuracy: 0.6000 - loss: 0.6941
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.6000 - loss: 0.6939
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6000 - loss: 0.6936
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6000 - loss: 0.6934
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6000 - loss: 0.6932
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6000 - loss: 0.6930
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6000 - loss: 0.6928
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6000 - loss: 0.6926
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6000 - loss: 0.6924
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6000 - loss: 0.6922
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6000 - loss: 0.6920
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6000 - l

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [14]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
# Note:  100 Epoch Result was Loss: 0.5410, Accuracy: 0.7667
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - 82ms/step - accuracy: 0.6429 - loss: 0.6750
Loss: 0.6750372052192688, Accuracy: 0.6428571343421936


Note:  This model does not provide much assistance in predicting outcome.

Future work:  The glacier retreat response could be slow to respond.  Try
shifting the data.

Future work:  Simpler models may be more applicable for this particular dataset.

Try a linear regression model with the glacier measurements instead of classification.


### Step 4: Save and export your model to a keras file, and name the file '`model_01.keras`.
Second Pass:  'model_01a.keras'
Third Pass:  'model_01b.keras'


In [15]:
# Set the model's file path
file_path = Path('saved_models/model_1.keras')

# Export your model to a keras file
nn.save(file_path)

---
### Predict Glacier Retreat by Using your Neural Network Model

Step 1: Reload the saved model.

In [16]:
# Set the model's file path
file_path = Path('saved_models/model_1.keras')

# Load the model to a new object
nn = tf.keras.models.load_model(file_path)

Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [17]:
# Make predictions with the test data
predictions = nn.predict(X_test_scaled,verbose=2)

# Display a sample of the predictions
predictions[0:5]

1/1 - 0s - 42ms/step


array([[0.5448486 ],
       [0.47592303],
       [0.47592303],
       [0.47592303],
       [0.47592303]], dtype=float32)

In [18]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns=["predictions"], data=predictions)
predictions_df["predictions"] = round(predictions_df["predictions"],0)
predictions_df

,predictions
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,1.0


### Step 4: Display a classification report with the y test data and predictions

In [19]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"].values))

              precision    recall  f1-score   support

           0       0.60      0.86      0.71         7
           1       0.75      0.43      0.55         7

    accuracy                           0.64        14
   macro avg       0.68      0.64      0.63        14
weighted avg       0.67      0.64      0.63        14



Note:  On 'model_1a.keras' with test_data=.2, accuracy declined significantly